In [71]:
import keras.backend as K
import tensorflow as tf
import keras
import keras.layers as layer
import numpy as np

dataset = tf.data.TextLineDataset("data/europarl-v7-light.en")


def custom_standardization(input_data):
    return tf.strings.lower(input_data)


t_layer = keras.layers.TextVectorization(standardize=custom_standardization, output_mode='int')
t_layer.adapt(dataset)

vocab_size = t_layer.vocabulary_size()

window_size = 2  # context window size
embed_size = 100

In [228]:
from keras.preprocessing.text import text_to_word_sequence
from keras.utils import np_utils

x_train = []
y_train = []

train = []

for s in dataset:
    split = tf.strings.split(s, sep=' ')

    for i in range(window_size, len(split) - window_size):
        xs = []
        for j in range(i - window_size, i):
            xs.append(split[j])
        for j in range(i + 1, min(i + window_size + 1, len(split))):
            xs.append(split[j])

        xs_join = ' '.join([str(x.numpy().decode('utf-8')) for x in xs])
        y_join = int(t_layer(split[i]).numpy())
        y_temp = np.zeros(vocab_size)
        y_temp[y_join] = 1
        x_train.append(str(xs_join))
        y_train.append(y_temp.tolist())

        train.append((xs_join, y_join))

In [234]:
print(x_train)
print(y_train)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [230]:
def generate_cbow():
    cbow = keras.Sequential()
    cbow.add(keras.layers.Input(shape=(1,), dtype=tf.string))
    cbow.add(t_layer)
    cbow.add(keras.layers.Embedding(vocab_size, embed_size, name="embedding"))
    cbow.add(keras.layers.Lambda(lambda x: K.mean(x, axis=-2), output_shape=(4, 100)))
    cbow.add(keras.layers.Dense(vocab_size, activation='softmax'))

    return cbow

In [236]:
model = generate_cbow()

model.build()
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_7 (TextV  (None, None)             0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, None, 100)         158600    
                                                                 
 lambda_19 (Lambda)          (None, 100)               0         
                                                                 
 dense_19 (Dense)            (None, 1586)              160186    
                                                                 
Total params: 318,786
Trainable params: 318,786
Non-trainable params: 0
_________________________________________________________________


In [256]:
model.fit(x_train, y_train, epochs=200)

Epoch 1/200
149/149 [==============================] - 0s 3ms/step - loss: 2.5357 - accuracy: 0.4874
Epoch 2/200
149/149 [==============================] - 0s 3ms/step - loss: 2.3501 - accuracy: 0.5296
Epoch 3/200
149/149 [==============================] - 0s 3ms/step - loss: 2.1819 - accuracy: 0.5640
Epoch 4/200
149/149 [==============================] - 0s 3ms/step - loss: 2.0290 - accuracy: 0.5986
Epoch 5/200
149/149 [==============================] - 0s 3ms/step - loss: 1.8882 - accuracy: 0.6292
Epoch 6/200
149/149 [==============================] - 1s 3ms/step - loss: 1.7571 - accuracy: 0.6604
Epoch 7/200
149/149 [==============================] - 1s 3ms/step - loss: 1.6359 - accuracy: 0.6937
Epoch 8/200
149/149 [==============================] - 1s 3ms/step - loss: 1.5241 - accuracy: 0.7199
Epoch 9/200
149/149 [==============================] - 0s 3ms/step - loss: 1.4186 - accuracy: 0.7475
Epoch 10/200
149/149 [==============================] - 0s 3ms/step - loss: 1.3214 - accura

In [198]:
model.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_7 (TextV  (None, None)             0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, None, 100)         158600    
                                                                 
 lambda_17 (Lambda)          (None, 100)               0         
                                                                 
 dense_17 (Dense)            (None, 1586)              160186    
                                                                 
Total params: 318,786
Trainable params: 318,786
Non-trainable params: 0
_________________________________________________________________


In [160]:
tok = ["Resumption of the session"]

In [128]:
xtemp = model.layers[0](tok)
xtemp.shape

TensorShape([1, 4])

In [129]:
xtemp = model.layers[1](xtemp)
xtemp.shape

TensorShape([1, 4, 100])

In [130]:
xtemp = model.layers[2](xtemp)
xtemp.shape

TensorShape([1, 100])

In [131]:
xtemp = model.layers[3](xtemp)
xtemp.shape

TensorShape([1, 1586])

In [268]:
predict = model.predict(['Resumption of the'])

1/1 [==============================] - 0s 43ms/step


In [269]:
t_layer.get_vocabulary()[np.argmax(predict)]

'group'